In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
from os.path import join as pjoin
from tqdm.notebook import tqdm
import plotly.graph_objects as go
from scipy.stats import pearsonr, spearmanr, zscore
from natsort import natsorted
import plotly.express as px

sys.path.append('/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis')
import circletrack_behavior as ctb
import circletrack_neural as ctn
import plotting_functions as pf

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
## Settings
dpath = '../../../MultiCon_Imaging/MultiCon_Imaging2/output/'
fig_path = '../../../MultiCon_Imaging/MultiCon_Imaging2/intermediate_figures/'
mouse_list = ['mc23', 'mc26']
session_list = [f'A{x}' for x in np.arange(1, 6)] + [f'B{x}' for x in np.arange(1, 6)] + [f'C{x}' for x in np.arange(1, 6)] + [f'D{x}' for x in np.arange(1, 6)]

In [3]:
results = {'mouse': [], 'session': [], 'num_cells': []}
for mouse in mouse_list:
    mpath = pjoin(dpath, f'aligned_minian/{mouse}/S')
    for session in os.listdir(mpath):
        spike_data = xr.open_dataset(pjoin(mpath, session))['S'] ## select S matrix
        results['mouse'].append(mouse)
        results['session'].append(spike_data.attrs['session_two'])
        results['num_cells'].append(spike_data.shape[0])
result_df = pd.DataFrame(results)
avg_cells = result_df.groupby(['session'], as_index=False).agg({'num_cells': ['mean', 'sem']})

In [6]:
fig = pf.custom_graph_template(x_title='Session', y_title='Number of Cells')
fig.add_trace(go.Scatter(x=avg_cells['session'], y=avg_cells['num_cells']['mean'], mode='lines+markers', marker_color='darkgrey',
                         error_y=dict(type='data', array=avg_cells['num_cells']['sem'])))
fig.show()